In [3]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/content/combined_financial_data_idx.csv')  # Correct path

# Show basic info
print(df.info())

# See first few rows
print(df.head())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89243 entries, 0 to 89242
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   symbol   89243 non-null  object 
 1   account  89243 non-null  object 
 2   type     89243 non-null  object 
 3   2020     83988 non-null  float64
 4   2021     84474 non-null  float64
 5   2022     83634 non-null  float64
 6   2023     69831 non-null  float64
dtypes: float64(4), object(3)
memory usage: 4.8+ MB
None
  symbol                                     account type          2020  \
0   AALI                            Accounts Payable   BS  7.702640e+11   
1   AALI                         Accounts Receivable   BS  7.658490e+11   
2   AALI                    Accumulated Depreciation   BS -1.092095e+13   
3   AALI                  Additional Paid In Capital   BS  3.878995e+12   
4   AALI  Allowance For Doubtful Accounts Receivable   BS -2.426100e+10   

           2021          2022          202

In [4]:
# Show basic info
print(df.info())

# See first few rows
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89243 entries, 0 to 89242
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   symbol   89243 non-null  object 
 1   account  89243 non-null  object 
 2   type     89243 non-null  object 
 3   2020     83988 non-null  float64
 4   2021     84474 non-null  float64
 5   2022     83634 non-null  float64
 6   2023     69831 non-null  float64
dtypes: float64(4), object(3)
memory usage: 4.8+ MB
None
  symbol                                     account type          2020  \
0   AALI                            Accounts Payable   BS  7.702640e+11   
1   AALI                         Accounts Receivable   BS  7.658490e+11   
2   AALI                    Accumulated Depreciation   BS -1.092095e+13   
3   AALI                  Additional Paid In Capital   BS  3.878995e+12   
4   AALI  Allowance For Doubtful Accounts Receivable   BS -2.426100e+10   

           2021          2022          202

In [5]:
# Check missing values
print(df.isnull().sum())


symbol         0
account        0
type           0
2020        5255
2021        4769
2022        5609
2023       19412
dtype: int64


In [7]:
# Drop rows with missing values
df = df.dropna()


In [8]:
# If there are 'Year' and 'Quarter' columns
df['Quarter'] = df['Quarter'].astype(str)
df['Year'] = df['Year'].astype(str)
df['Date'] = df['Year'] + ' Q' + df['Quarter']

# Now convert to datetime if possible
# Or just leave it as Year+Quarter format for trend analysis


KeyError: 'Quarter'

In [9]:
# See column names
print(df.columns)


Index(['symbol', 'account', 'type', '2020', '2021', '2022', '2023'], dtype='object')


In [10]:
# Reshape the data
df_melted = pd.melt(df, id_vars=['symbol', 'account', 'type'],
                    value_vars=['2020', '2021', '2022', '2023'],
                    var_name='year', value_name='value')

# Check result
print(df_melted.head())


  symbol                                     account type  year         value
0   AALI                            Accounts Payable   BS  2020  7.702640e+11
1   AALI                         Accounts Receivable   BS  2020  7.658490e+11
2   AALI                    Accumulated Depreciation   BS  2020 -1.092095e+13
3   AALI                  Additional Paid In Capital   BS  2020  3.878995e+12
4   AALI  Allowance For Doubtful Accounts Receivable   BS  2020 -2.426100e+10


In [13]:
# Convert 'year' to integer
df_melted['year'] = df_melted['year'].astype(int)

# Convert 'value' to numeric (if not already)
df_melted['value'] = pd.to_numeric(df_melted['value'], errors='coerce')


In [14]:
# Check missing values
print(df_melted.isnull().sum())


symbol     0
account    0
type       0
year       0
value      0
dtype: int64


In [16]:
# Save the cleaned and melted dataset
df_melted.to_csv('/content/cleaned_financial_data.csv', index=False)


In [17]:
# Filter Revenue
revenue_df = df_melted[df_melted['account'].str.contains('Revenue', case=False)]

# Filter Net Profit
net_profit_df = df_melted[df_melted['account'].str.contains('Net Income|Net Profit', case=False, na=False)]

# Filter Total Assets
assets_df = df_melted[df_melted['account'].str.contains('Total Asset', case=False, na=False)]

# Filter Total Liabilities
liabilities_df = df_melted[df_melted['account'].str.contains('Total Liabilities', case=False, na=False)]


In [18]:
print(revenue_df.head())
print(net_profit_df.head())
print(assets_df.head())
print(liabilities_df.head())


    symbol                     account type  year         value
24    AALI             Cost Of Revenue   IS  2020  1.584415e+13
88    AALI           Operating Revenue   IS  2020  1.880704e+13
111   AALI  Reconciled Cost Of Revenue   IS  2020  1.584415e+13
135   AALI               Total Revenue   IS  2020  1.880704e+13
163   ABBA             Cost Of Revenue   IS  2020  8.206386e+10
   symbol                                            account type  year  \
68   AALI                                         Net Income   IS  2020   
69   AALI                     Net Income Common Stockholders   IS  2020   
70   AALI                   Net Income Continuous Operations   IS  2020   
71   AALI  Net Income From Continuing And Discontinued Op...   IS  2020   
72   AALI  Net Income From Continuing Operation Net Minor...   IS  2020   

           value  
68  8.330900e+11  
69  8.330900e+11  
70  8.937790e+11  
71  8.330900e+11  
72  8.330900e+11  
    symbol       account type  year         value
1

In [19]:
# Pivot Revenue
revenue_pivot = revenue_df.pivot_table(index='symbol', columns='year', values='value')

# Calculate Revenue Growth
revenue_pivot['Revenue Growth (%)'] = ((revenue_pivot[2023] - revenue_pivot[2022]) / revenue_pivot[2022]) * 100

# View it
print(revenue_pivot[['Revenue Growth (%)']].head())


year    Revenue Growth (%)
symbol                    
AALI             -2.799290
ABBA             26.539377
ABDA             15.484082
ABMM              9.488684
ACES             12.361398


In [20]:
# Merge Revenue and Net Profit for 2023
revenue_2023 = revenue_df[revenue_df['year'] == 2023][['symbol', 'value']].rename(columns={'value': 'revenue_2023'})
profit_2023 = net_profit_df[net_profit_df['year'] == 2023][['symbol', 'value']].rename(columns={'value': 'profit_2023'})

merged_profit = pd.merge(profit_2023, revenue_2023, on='symbol')

# Calculate Net Profit Margin
merged_profit['Net Profit Margin (%)'] = (merged_profit['profit_2023'] / merged_profit['revenue_2023']) * 100

# View
print(merged_profit[['symbol', 'Net Profit Margin (%)']].head())


  symbol  Net Profit Margin (%)
0   AALI               5.874419
1   AALI               5.089771
2   AALI               5.874419
3   AALI               5.089771
4   AALI               5.874419


In [21]:
# Merge Assets and Liabilities for 2023
assets_2023 = assets_df[assets_df['year'] == 2023][['symbol', 'value']].rename(columns={'value': 'assets_2023'})
liabilities_2023 = liabilities_df[liabilities_df['year'] == 2023][['symbol', 'value']].rename(columns={'value': 'liabilities_2023'})

merged_debt = pd.merge(liabilities_2023, assets_2023, on='symbol')

# Calculate Debt to Asset Ratio
merged_debt['Debt to Asset Ratio'] = merged_debt['liabilities_2023'] / merged_debt['assets_2023']

# View
print(merged_debt[['symbol', 'Debt to Asset Ratio']].head())


  symbol  Debt to Asset Ratio
0   AALI             0.217714
1   ABBA             1.390180
2   ABDA             0.418694
3   ABMM             0.648105
4   ACES             0.202092


In [22]:
# Merge Revenue Growth and Net Profit Margin
kpi_merged = pd.merge(revenue_pivot[['Revenue Growth (%)']].reset_index(),
                      merged_profit[['symbol', 'Net Profit Margin (%)']], on='symbol')

# Merge with Debt to Asset Ratio
kpi_merged = pd.merge(kpi_merged,
                      merged_debt[['symbol', 'Debt to Asset Ratio']], on='symbol')

# View Final KPI table
print(kpi_merged.head())


  symbol  Revenue Growth (%)  Net Profit Margin (%)  Debt to Asset Ratio
0   AALI            -2.79929               5.874419             0.217714
1   AALI            -2.79929               5.089771             0.217714
2   AALI            -2.79929               5.874419             0.217714
3   AALI            -2.79929               5.089771             0.217714
4   AALI            -2.79929               5.874419             0.217714


In [23]:
# Save final KPI table
kpi_merged.to_csv('/content/final_financial_kpis.csv', index=False)
